# Triangular mesh from obj file shown using matplotlib
A small example demonstrating some operations with triangular meshes: loading an obj file, crating mplot3d mesh object, computing shading, assigning colors to faces, and visualizing.  
Author: vand@dtu.dk, 2020

In [ ]:
import numpy as np
import urllib.request
import mpl_toolkits.mplot3d.art3d 
import matplotlib.pyplot as plt
import matplotlib.colors 

def load_obj(filename):
    """ Load triangle mesh from obj file.
    """
    vertices = []
    faces = []
    with open(filename, 'r') as f:        
        for line in f:
            if line and line[0]=='v':
                vertex = np.array(line[2:].split(' '), dtype=float)                
                vertices.append(vertex)
            if line and line[0]=='f':
                face = np.array(line[2:].split(' '), dtype=int)  
                faces.append(face)
    return np.array(vertices), np.array(faces)

def load_url_obj(url):
    """ Load triangle mesh from online obj file.
    """
    vertices = []
    faces = []
    with urllib.request.urlopen(url) as response:
        for bline in response:
            line = bline.decode()
            if line and line[0]=='v':
                vertex = np.array(line[2:].split(' '), dtype=float)                
                vertices.append(vertex)
            if line and line[0]=='f':
                face = np.array(line[2:].split(' '), dtype=int)  
                faces.append(face)
    return np.array(vertices), np.array(faces)

In [ ]:
# loading from url (of file, commented out) and slightly adjusting the mesh
url = 'https://graphics.stanford.edu/~mdfisher/Data/Meshes/bunny.obj'
vertices, faces = load_url_obj(url)
#filename = 'bunny.obj'
#vertices, faces = load_obj(filename)
faces -= 1 # 0-indexing
vertices -= vertices.mean(axis=0) # vertices in origo
vertices /= np.abs(vertices).max() # vertices in box [-1 1]^3

# computing face shading intensity based on face normals
ls = matplotlib.colors.LightSource(azdeg=0, altdeg=0)
normals = np.cross(vertices[faces[:,1]]-vertices[faces[:,0]],vertices[faces[:,2]]-vertices[faces[:,0]])
normals /= np.sqrt((normals**2).sum(axis=1)).reshape(-1,1) # normalizing
intensity = ls.shade_normals(normals)

# basic mesh color, same for all faces, here red-ish
color = np.tile([[[1,0.5,0]]],(normals.shape[0],1,1))

# blending face colors and face shading intensity
rgb = ls.blend_hsv(rgb = color, intensity=intensity.reshape(-1,1,1)) 

# adding alpha value, may be left out
rgba = np.concatenate((rgb,0.9*np.ones(shape=(rgb.shape[0],1,1))),axis=2) 

# creating mesh with given face colors
mesh = mpl_toolkits.mplot3d.art3d.Poly3DCollection(vertices[faces])
mesh.set_facecolor(rgba.reshape(-1,4))

In [ ]:
# and now -- visualization!
fig = plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
ax.add_collection3d(mesh)
ax.view_init(azim=-90, elev=120) # rather limited control of camera, where's up-vector?
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(-1,1)
ax.set_axis_off()
plt.show()